In [4]:

from google.cloud import aiplatform
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform_v1
import pandas as pd

film_df = pd.read_json("../data/film_01.json", lines=True)

aiplatform.init(
    project='vidio-quiz-prod',
    location='asia-southeast1',
    staging_bucket='gs://genai_hackathon_2024',
)

API_ENDPOINT="574946744.asia-southeast1-328583281153.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/328583281153/locations/asia-southeast1/indexEndpoints/8923530817913749504"
DEPLOYED_INDEX_ID="film_metadata"
client_options = {
  "api_endpoint": API_ENDPOINT
}

In [2]:
model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")

In [5]:
# query_embed = model.get_embeddings(["Saya pengen nonton film action, apa ya yang cocok"])
# query_embed = model.get_embeddings(["Actors: dian sastrowardoyo"])
# query_embed = model.get_embeddings(["vidio original yang actor nya dian sastro"])
query_embed = model.get_embeddings(["ratu adil"])

vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=query_embed[0].values
)
query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  # The number of nearest neighbors to be retrieved
  neighbor_count=10
)

request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  # Request can have multiple queries
  queries=[query],
  return_full_datapoint=False,
)

# Execute the request
response = vector_search_client.find_neighbors(request)
ids = [int(neighbors.datapoint.datapoint_id) for neighbors in response.nearest_neighbors[0].neighbors]

search_result_df = film_df[film_df["id"].isin(ids)]
search_result_df 

,id,film_title,group_name_l1,group_name_l2,group_name_l3,film_main_genre,film_genres,film_directors,film_actors,country_group,search_text,embedding
1178,6147,Drama Ratu Drama,Series,Vidio Original,NaN,drama,"adaptation,comedy,drama",aco tenriyagelli,"aulia deas,bukie b. mansyur,enzy storia,ibrahi...",Indonesia,Title: Drama Ratu Drama\nGroup: Series > Vidio...,"[0.0099508706, -0.0031534047, 0.0093631763, 0...."
1398,9538,Rahasia Cinta Bawang Putih,Movies,FTV,NaN,drama,"drama,romance",rizal basri,"lavicky nicholas,rachquel nesia",Indonesia,Title: Rahasia Cinta Bawang Putih\nGroup: Movi...,"[-0.0054098526, -0.0268836562, -0.0222054999, ..."
1873,9372,Ratu Adil,Series,Vidio Original,NaN,crime,"action,crime,drama","ginanti rona,tommy dewo","abdurrahman arif,andri mashadi,budi ros,dian s...",Indonesia,Title: Ratu Adil\nGroup: Series > Vidio Origin...,"[-0.012067695100000001, -0.0112606818, 0.00795..."
3330,5735,Udah Go Public Tapi Masih Shy Shy Cat,Movies,FTV,NaN,drama,"drama,romance",ninos joned,"agatha valerie,rendy kjaernett",Indonesia,Title: Udah Go Public Tapi Masih Shy Shy Cat\n...,"[0.0198711473, -0.036032822, 0.007560154400000..."
3346,2571,LIDA 2021,Entertainment,Talent Search,NaN,dangdut,"dangdut,variety shows",None,"gilang dirga,inul daratista,irfan hakim,jirayu...",Indonesia,Title: LIDA 2021\nGroup: Entertainment > Talen...,"[0.027185872200000002, -0.0168020986, -0.01227..."
3662,8395,Dukaku Adalah Kebahagiaan Suamiku,Movies,FTV,NaN,drama,"drama,romance",olla ata adonara,"kenya nindya,rifky al habsy",Indonesia,Title: Dukaku Adalah Kebahagiaan Suamiku\nGrou...,"[0.0423445515, 0.0025979702000000003, -0.01621..."
3855,6186,Pantang Pulang Sebelum Halal,Movies,FTV,NaN,drama,"comedy,drama,romance",m. haikal,"chand kelvin,fay nabila",Indonesia,Title: Pantang Pulang Sebelum Halal\nGroup: Mo...,"[0.0065556844, -0.0310590751, -0.0130229592, 0..."
4554,4562,Dodol Perekat Cinta,Movies,FTV,NaN,drama,"drama,romance",rievy indriasari,"derry drajat,eza gionino",Indonesia,Title: Dodol Perekat Cinta\nGroup: Movies > FT...,"[0.026392201, -0.0094242692, -0.0180741008, 0...."
5089,2298,Sepotong Cinta Roti Maryam,Movies,FTV,NaN,drama,"drama,romance",m. haikal,"adinda azani,arya saloka,rendy kjaernett",Indonesia,Title: Sepotong Cinta Roti Maryam\nGroup: Movi...,"[0.017904026400000002, 0.0104972031, -0.015628..."
5644,731,Seharum Bunga Cinta Ratu Ayu,Movies,FTV,NaN,drama,"comedy,drama,romance",otoy witoyo,"faradilla yoshi,kiki farrel,rachquel nesia,rez...",Indonesia,Title: Seharum Bunga Cinta Ratu Ayu\nGroup: Mo...,"[0.014026827200000001, 0.0006056986, 0.0326167..."


In [106]:
[print(search_result,end="\n\n") for search_result in search_result_df["search_text"].values.tolist()]

Title: Cinta Seorang Anak Penyadap Pohon Karet Yang Bercita-cita Membangun Rumah Untuk Ibunya
Group: Movies > FTV > nan
Genres: drama,drama,religi
Directors: aca hasanuddin
Actors: ratu idola,sridevi
Country: Indonesia

Title: Drama Ratu Drama
Group: Series > Vidio Original > nan
Genres: drama,adaptation,comedy,drama
Directors: aco tenriyagelli
Actors: aulia deas,bukie b. mansyur,enzy storia,ibrahim risyad,rachel amanda,randy danistha,teuku rifnu wikana,yayu a.w. unru
Country: Indonesia

Title: Ratu Adil
Group: Series > Vidio Original > nan
Genres: crime,action,crime,drama
Directors: ginanti rona,tommy dewo
Actors: abdurrahman arif,andri mashadi,budi ros,dian sastrowardoyo,donny damara,hana malasan,ira wibowo,khiva iskak,muhammad khan,nino fernandez
Country: Indonesia

Title: Ratu Comblang Galau Akut
Group: Movies > FTV > nan
Genres: drama,drama,romance
Directors: chand parwez servia
Actors: adzana bing slamet,dimas aditya
Country: Indonesia

Title: Akan Aku Treat Kamu Bagaikan Ratu
Gr

[None, None, None, None, None, None, None, None, None, None]